In [13]:
import psycopg2
import psycopg2.extras
import csv
from config import params

conn = psycopg2.connect(**params)

with conn.cursor() as cursor:
    with open('/Users/ezequielesparza/TKH-Social-Media-Engagement-Analytics/instagram_analytics/instagram.sql', 'r') as schema:
        queries = schema.read()
        print(queries)
        cursor.execute(queries)


    try:
        conn.commit()
    except Exception as e:
        print(e)
        conn.rollback()


DROP TABLE IF EXISTS market.instagram CASCADE ;
DROP TABLE IF EXISTS market.instagram_hashtag CASCADE;
DROP TABLE IF EXISTS market.instagram_hash_post CASCADE;



CREATE TABLE market.instagram(
    id VARCHAR(255) PRIMARY KEY,
    type VARCHAR(255), 
    commentsCount FLOAT,
    likesCount FLOAT,
    ig_timestamp timestamp,
    videoViewCount FLOAT,
    videoPlayCount FLOAT,
    videoDuration FLOAT    
);
CREATE TABLE market.instagram_hashtag(
    index_num int PRIMARY KEY,
    hashtag VARCHAR(255)
);
CREATE TABLE market.instagram_hash_post(
    row_index INT PRIMARY KEY,
    ID VARCHAR(255) REFERENCES market.instagram(id),
    hashtag_index int REFERENCES market.instagram_hashtag(index_num)
);



In [15]:
conn = psycopg2.connect(**params)

with conn.cursor() as cursor:
    
    with open('/Users/ezequielesparza/TKH-Social-Media-Engagement-Analytics/data/without_hashtags.csv', 'r') as f:    
        cmd = 'COPY market.instagram FROM STDIN WITH (FORMAT CSV, HEADER TRUE)'
        cursor.copy_expert(cmd, f)

    with open('/Users/ezequielesparza/TKH-Social-Media-Engagement-Analytics/data/just_hashtags', 'r') as f:    
        cmd = 'COPY market.instagram_hashtag FROM STDIN WITH (FORMAT CSV, HEADER TRUE)'
        cursor.copy_expert(cmd, f)

    with open('/Users/ezequielesparza/TKH-Social-Media-Engagement-Analytics/data/primary_table.csv', 'r') as f:    
        cmd = 'COPY market.instagram_hash_post FROM STDIN WITH (FORMAT CSV, HEADER TRUE)'
        cursor.copy_expert(cmd, f)


    
    conn.commit()